# Hackaton Decathlon

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix, recall_score, roc_auc_score
from imblearn.metrics import specificity_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.neural_network import MLPClassifier

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
usuarios= pd.read_csv('users_IT.csv', index_col=0, engine='python')
productos= pd.read_csv('prods_IT.csv', index_col=0, engine='python')
recomendations_train= pd.read_csv('recomendations_train_IT.csv', index_col=0, engine='python')
recomendations_test= pd.read_csv('recomendations_test.csv', index_col=0, engine='python')

In [3]:
usuarios.head()

,complete_name,age,address
person_id,,,
d0eb7350-ce10-42a2-bab1-cb66319bdfa6,Leah Wright,26,"5942 Phillip Ridge\nWest Hannah, WA 76733"
06ada2a9-ec6e-4306-b83b-33af4489fca0,Jonathan Hill,26,"390 Shelley Ways\nBoydburgh, NY 48020"
acf71426-adca-4f48-87d6-4978dc538448,Anne Dougherty,28,USCGC Thompson\nFPO AE 51140
07017851-e9cc-40d2-a0f3-e82819761e72,Lydia Hall,41,"5511 Miller Dam\nWest Michael, UT 03004"
d116bb8a-43df-42a5-9575-5af33e088ec9,Brianna Washington,28,"70072 John Keys\nNew Amandaport, OH 85289"


In [23]:
productos.head()

,weblabel,brand,catchline,productnature,family,Review-AverageRating,DisplayPrice-Amount,ListPrice-Amount,precio_dif
modeild,,,,,,,,,
8071204,REGULADOR DE RED DE TENIS,ARTENGO,Indispensable para una buena altura de red.,REDES,EQUIPAMIENTO COACH/CLUB,5.0,9.99,9.99,False
8071566,PELOTA DE TENIS POLIVALENTE TB920 *4 AMARILLO,ARTENGO,¡Durabilidad y control!Pelota oficial del torn...,Pelota de tenis,PELOTAS TENIS,4.5,5.99,5.99,False
8071569,PELOTA DE TENIS TB530 *4 AMARILLO,ARTENGO,¡Pelota de competición a bajo precio!Nuevo emp...,Pelota de tenis,PELOTAS TENIS,4.5,4.49,4.49,False
8072596,CORDAJE DE TENIS BABOLAT MULTIFILAMENTOS XCEL ...,BABOLAT,Este cordaje de tenis multifilamentos Babolat ...,Cordaje raqueta,CORDAJES TENIS,4.7,14.99,14.99,False
8072605,Cordaje Tenis Monofilamento Big Banger Alu Pow...,LUXILON,Este cordaje de tenis monofilamento Luxilon Al...,Cordaje raqueta,CORDAJES TENIS,4.3,14.99,14.99,False


In [27]:
recomendations_train

,time_reco,id_sender,id_reciever,id_product,product_bought
reco_id,,,,,
5dc916dc-6fa3-488f-8ad2-fd14100fa113,2008-02-09 17:58:24,5565ac1e-4c2a-4fd8-b990-7a11d594dc0e,13a6c635-2f20-4e22-9e76-c3761d1111be,8605674,1
e9b6a2bb-58d4-4e62-8fda-239da00e42a1,1995-06-17 04:27:17,4c1e438a-2b8c-4f84-a22a-869eea346661,84f27bd9-0d26-4f84-b585-cca0e4bbd1d5,8603017,1
91a31be2-e71f-436a-917e-1ac65cc9ef2e,2002-04-10 19:26:53,51c3fb8f-c26f-4808-a69a-3f728256cf7a,8c333f9b-af38-4a0c-8bdc-428ffa061312,8763489,1
b69901f9-c70b-4133-be43-cc6116f1c8fd,2015-07-31 11:41:37,45e9a922-e602-4e8f-be77-d6486779ae0f,5f9dfbb7-8bca-494a-802e-f11f12c9906a,8607307,1
0ccdd3e7-a1cb-43fd-814d-a501278c9f9c,2001-04-06 20:20:43,250343f4-b757-4c3f-aaea-d7d3ddb14744,34d40a00-18f9-4fa5-9973-ffd5d8663c6f,8372995,1
...,...,...,...,...,...
b15fcb43-75a5-4a7a-904e-1707686d57a1,1991-01-12 11:28:16,e3d1e9ab-01fb-4ade-ae58-8449f8fbe742,6d1012bc-d57d-4095-87f9-8ef9767987d5,8561587,0
4b5d39cf-2b39-4b84-8052-d94b14193f69,2004-04-21 20:41:31,807a90a6-3103-4cef-944a-304a663b58e1,30edad71-d3f3-479f-87ad-f093880ccf80,8667378,1
50349b56-4dbe-4da4-8145-4abd1f485b94,1977-01-02 22:35:35,81bf0b99-99aa-4211-92be-2315810ee347,cc410d52-c17b-4b93-b07e-e217a75738c6,8487421,1


In [26]:
#Las recomendaciones están balanceadas
recomendations_train.product_bought.value_counts(normalize=True)

1    0.614571
0    0.385429
Name: product_bought, dtype: float64

In [3]:
recomendation_product=pd.read_csv('recomendation_products2.csv',index_col=0, engine='python')


In [5]:
recomendation_product.head()


,id_sender,id_reciever,id_product,product_bought,year,month,weblabel,brand,catchline,productnature,family,Review-AverageRating,DisplayPrice-Amount,ListPrice-Amount,precio_dif
reco_id,,,,,,,,,,,,,,,
5dc916dc-6fa3-488f-8ad2-fd14100fa113,5565ac1e-4c2a-4fd8-b990-7a11d594dc0e,13a6c635-2f20-4e22-9e76-c3761d1111be,8605674,1,2008,9,Zapatillas Squash Salming Viper 5 Hombre Amari...,SALMING,"La Viper es una zapatilla rápida, flexible y d...",ZAPATILLAS,ZAPATILLA SQUASH,5.0,79.99,99.99,True
e9b6a2bb-58d4-4e62-8fda-239da00e42a1,4c1e438a-2b8c-4f84-a22a-869eea346661,84f27bd9-0d26-4f84-b585-cca0e4bbd1d5,8603017,1,1995,17,Camiseta de bádminton manga corta Mujer Perfly...,PERFLY,Primera camiseta por precio pensada para el bá...,CAMISETAS MANGA CORTA,Ropa badminton mujer,4.6,9.99,9.99,False
91a31be2-e71f-436a-917e-1ac65cc9ef2e,51c3fb8f-c26f-4808-a69a-3f728256cf7a,8c333f9b-af38-4a0c-8bdc-428ffa061312,8763489,1,2002,10,Zapatillas tenis Hombre Revolt Pro 4.0 Negro A...,HEAD,Estas zapatillas de tenis HEAD REVOLT PRO 4.0 ...,ZAPATILLAS,CALZADO HOMBRE MULTICANCHA EXPERTO,5.0,124.99,124.99,False
b69901f9-c70b-4133-be43-cc6116f1c8fd,45e9a922-e602-4e8f-be77-d6486779ae0f,5f9dfbb7-8bca-494a-802e-f11f12c9906a,8607307,1,2015,31,Zapatillas de pádel Mujer Kuikma PS 990 Dynami...,KUIKMA,Estas zapatillas de pádel te aportarán el dina...,ZAPATILLAS,CALZADO PADEL,4.4,64.99,64.99,False
0ccdd3e7-a1cb-43fd-814d-a501278c9f9c,250343f4-b757-4c3f-aaea-d7d3ddb14744,34d40a00-18f9-4fa5-9973-ffd5d8663c6f,8372995,1,2001,6,Zapatillas de tenis Mujer TS 100 blanco,ARTENGO,Zapatillas de tenis concebidas para dar los pr...,ZAPATILLAS,CALZADO MUJER MULTICANCHA INIC/PERFECC,4.5,12.99,12.99,False


In [49]:
recomendation_product.describe()

,id_product,product_bought,year,month,Review-AverageRating,DisplayPrice-Amount,ListPrice-Amount
count,7.000000e+03,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,8.396678e+06,0.614571,1995.672286,15.682571,3.807957,41.580111,43.177433
std,1.246082e+06,0.486731,15.064249,8.736827,1.651877,76.079668,76.618422
min,8.606340e+05,0.000000,1970.000000,1.000000,0.000000,0.990000,0.990000
25%,8.552495e+06,0.000000,1983.000000,8.000000,4.100000,11.990000,11.990000
50%,8.607990e+06,1.000000,1995.000000,16.000000,4.500000,22.990000,24.990000
75%,8.667576e+06,1.000000,2009.000000,23.000000,4.700000,54.990000,54.990000
max,8.808220e+06,1.000000,2022.000000,31.000000,5.000000,1349.990000,1349.990000


In [50]:
recomendation_product.id_product.value_counts()

8649223    40
8665808    40
8554892    36
8598789    35
8605668    34
           ..
8554947     2
8561587     2
8519699     2
8397173     1
8623050     1
Name: id_product, Length: 692, dtype: int64

In [52]:
recomendation_product.productnature.value_counts().shape

(75,)

In [4]:
data=recomendation_product[['product_bought', 'brand', 'productnature', 'family', 'Review-AverageRating', 'precio_dif']]
data

,product_bought,brand,productnature,family,Review-AverageRating,precio_dif
reco_id,,,,,,
5dc916dc-6fa3-488f-8ad2-fd14100fa113,1,SALMING,ZAPATILLAS,ZAPATILLA SQUASH,5.0,True
e9b6a2bb-58d4-4e62-8fda-239da00e42a1,1,PERFLY,CAMISETAS MANGA CORTA,Ropa badminton mujer,4.6,False
91a31be2-e71f-436a-917e-1ac65cc9ef2e,1,HEAD,ZAPATILLAS,CALZADO HOMBRE MULTICANCHA EXPERTO,5.0,False
b69901f9-c70b-4133-be43-cc6116f1c8fd,1,KUIKMA,ZAPATILLAS,CALZADO PADEL,4.4,False
0ccdd3e7-a1cb-43fd-814d-a501278c9f9c,1,ARTENGO,ZAPATILLAS,CALZADO MUJER MULTICANCHA INIC/PERFECC,4.5,False
...,...,...,...,...,...,...
b15fcb43-75a5-4a7a-904e-1707686d57a1,0,ARTENGO,Cordaje raqueta,CORDAJES TENIS,5.0,False
4b5d39cf-2b39-4b84-8052-d94b14193f69,1,ARTENGO,Pantalón corto,Textil tenis mujer,4.6,False
50349b56-4dbe-4da4-8145-4abd1f485b94,1,ARTENGO,Pantalón corto,Textil tenis hombre,4.7,False


In [5]:
cat_vars=['brand', 'productnature', 'family']
data = pd.get_dummies(data, columns= cat_vars)

In [56]:
data.head()

,product_bought,Review-AverageRating,precio_dif,brand_ADIDAS,brand_ARAMBILLET,brand_ARTENGO,brand_ASICS,brand_BABOLAT,brand_BOYAUDERIE DE L'EST,brand_BULL PADEL,brand_CORNILLEAU,brand_DROP SHOT,brand_DUNLOP,brand_EUSKALDUNA,brand_FZ FORZA,brand_GEOLOGIC,brand_HEAD,brand_IKUS,brand_JOOLA,brand_K-SWISS,brand_KARAKAL,brand_KUIKMA,brand_MIZUNO,brand_NO BRAND,brand_OPFEEL,brand_PERFLY,brand_PONGORI,brand_PUMA,brand_REEBOK,brand_RSTA,brand_SALMING,brand_SCHILDKRÖT,brand_SHOCK OUT,brand_SPEEDMINTON,brand_TALBOT TORRO,brand_TECNIFIBRE,brand_TIBHAR,brand_TUBOPLUS,brand_UNICORN,brand_URBALL,brand_VICTOR,brand_WILSON,brand_YASAKA,brand_YONEX,productnature_Accesorio de aprendizaje,productnature_Antivibrador,productnature_BOLSAS DEPORTE,productnature_BOTAS,productnature_BOTEROS,productnature_Ball net,...,family_MADERAS/REVESTIMIENTOS/ACCESORIOS TDM,family_MESAS TDM CUALQUIER TIEMPO,family_MESAS TDM INTERIOR,family_MESAS/ACCESORIOS TDM CLUB,family_ONE WALL,family_PADEL CLUB,family_PALAS PÁDEL ADULTO,family_PALETA GOMA MACIZA,family_PEDIDO DIRECTO OTROS PELOTA,family_PEDIDOS DIRECTOS,family_PELOTAS PÁDEL,family_PELOTAS TDM,family_PELOTAS TENIS,family_PICKLEBALL,family_PÁDEL JÚNIOR,family_RAQUETAS ADULTO INICIACIÓN/PERFECCIONAM,family_RAQUETAS BÁDMINTON ADULTO,family_RAQUETAS BÁDMINTON JÚNIOR,family_RAQUETAS PING PONG INTERIOR,family_RAQUETAS SQUASH ADULTO,family_RAQUETAS SQUASH NIÑOS,family_RAQUETAS TENIS JÚNIOR,family_RAQUETTES BADMINTON ADULTE CONFIRME,family_RAQUETTES BADMINTON ADULTE EXPERT,family_RAQUETTES TDT CONFIRMÉ,family_REDES/POSTES BÁDMINTON,family_ROLLNET/MESAS PEQUEÑA SUPERFICIE,family_Raquetas adulto experto,family_Ropa badminton hombre,family_Ropa badminton mujer,family_Ropa badminton niños,family_SPEEDBALL,family_SPV TENIS,family_SQUASH 57 RAQUETA,family_TABLES TDT FREE DÉBUTANT,family_TALLER SQUASH,family_TEXTIL PÁDEL,family_Tenis ocasional,family_Textil tenis hombre,family_Textil tenis mujer,family_Textil tenis niños,family_Textil tenis tiempo frío hombre,family_Textil tenis tiempo frío mujer,family_Textil tenis tiempo frío niños,family_VOLANTS PLUMES,family_Volantes Badminton,family_ZAPATILLA SQUASH,family_ZAPATILLAS BÁDMINTON / SQUASH HOMBRE,family_ZAPATILLAS BÁDMINTON/SQUASH JÚNIOR,family_ZAPATILLAS TDM
reco_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5dc916dc-6fa3-488f-8ad2-fd14100fa113,1,5.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
e9b6a2bb-58d4-4e62-8fda-239da00e42a1,1,4.6,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
91a31be2-e71f-436a-917e-1ac65cc9ef2e,1,5.0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
b69901f9-c70b-4133-be43-cc6116f1c8fd,1,4.4,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0ccdd3e7-a1cb-43fd-814d-a501278c9f9c,1,4.5,False,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
#Separamos las variables del target
features= list(data.columns)[1:]
X=data[features]
y=data.product_bought


In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 5)

In [78]:
models = {}                                #Diccionario con los modelos a testear
models['Logistic Regression'] = LogisticRegression()
models['Decision Tree'] = DecisionTreeClassifier()
models['Random Forest'] = RandomForestClassifier()
models['AdaBoostClassifier'] = AdaBoostClassifier(random_state=0)
models['GradientBoostingClassifier'] = GradientBoostingClassifier(random_state=0)
models['K-Nearest Neighbor'] = KNeighborsClassifier()
models['Suport Vector Machine'] = SVC(random_state=0)

accuracy, precision, recall, f1, roc = {}, {}, {}, {}, {}

for key in models.keys():
    
    models[key].fit(X_train, y_train)
       
    predictions = models[key].predict(X_test)				 # Prediction 
       
    accuracy[key] = accuracy_score(predictions, y_test)	 		# Calculate Metrics
    precision[key] = precision_score(predictions, y_test)
    recall[key] = recall_score(predictions, y_test)
    f1[key] = f1_score(predictions, y_test)
    roc[key] = roc_auc_score(predictions, y_test)
    
metricas = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall'])	#Crea DF métricas
metricas['Accuracy'] = accuracy.values()
metricas['Precision'] = precision.values()
metricas['Recall'] = recall.values()
metricas['F1'] = f1.values()
metricas['Roc_Auc'] = roc.values()
metricas


C:\Users\joseb\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Accuracy,Precision,Recall,F1,Roc_Auc
Logistic Regression,0.708571,0.914062,0.712174,0.800587,0.702087
Decision Tree,0.680714,0.837054,0.713606,0.770416,0.647634
Random Forest,0.684286,0.861607,0.708257,0.777442,0.654128
AdaBoostClassifier,0.705000,0.901786,0.713151,0.796451,0.691781
GradientBoostingClassifier,0.707143,0.910714,0.712042,0.799216,0.698541
K-Nearest Neighbor,0.644286,0.765625,0.704312,0.733690,0.605677
Suport Vector Machine,0.714286,0.936384,0.709814,0.807507,0.724173


In [76]:
#Calculamos la matriz de confusión de los modelos
print('Matriz de confusión de los modelos:')
for key in models.keys():
    print(key)
    print(confusion_matrix(y_test, models[key].predict(X_test)))
    print()

Matriz de confusión de los modelos:
Logistic Regression
[[173 331]
 [ 77 819]]

Decision Tree
[[203 301]
 [146 750]]

Random Forest
[[189 315]
 [128 768]]

AdaBoostClassifier
[[179 325]
 [ 88 808]]

K-Nearest Neighbor
[[216 288]
 [210 686]]

Suport Vector Machine
[[161 343]
 [ 57 839]]



In [81]:
#Seleccionamos los parámetros a optimizar con GridSearch
parameters ={'C': [0.1,1, 10, 100, 1000], 
              }
#Definimos el GridSearch
svm = GridSearchCV(estimator = SVC(random_state=0), param_grid = parameters, scoring = 'f1')

#Entrenamos el GridSearch
svm.fit(X_train, y_train)

GridSearchCV(estimator=SVC(random_state=0),
             param_grid={'C': [0.1, 1, 10, 100, 1000]}, scoring='f1')

In [82]:
svm.best_estimator_

SVC(C=1, random_state=0)

# Entrenamos con todo el modelo

In [7]:
#Entrenamos el modelo con todo el train
svm_final=SVC(random_state=0)
svm_final.fit(X, y)

SVC(random_state=0)

In [15]:
test_transformado=pd.read_csv('recomendation_product3.csv',index_col=0, engine='python')

In [16]:
data_test=test_transformado[['brand', 'productnature', 'family', 'Review-AverageRating', 'precio_dif']]

In [17]:
cat_vars=['brand', 'productnature', 'family']
data_test = pd.get_dummies(data_test, columns= cat_vars)

In [18]:
set(X)-set(data_test.columns)

{'productnature_Funda de protección', 'productnature_Suelas de billar'}

In [19]:
data_test['productnature_Funda de protección']=0
data_test['productnature_Suelas de billar']=0

In [20]:
resultado = svm_final.predict(data_test)

C:\Users\joseb\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [21]:
resultado

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [22]:
#Generamos un dataframe con los resultados
resultado_df=pd.DataFrame({'Resultado': resultado})
resultado_df.head()

,Resultado
0,0
1,0
2,0
3,1
4,1


In [23]:
#Creamos archivo con los resultados
resultado_df.to_csv('resultado.csv', index = False )